In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# for data load
import os

# for reading and processing images
import imageio
from PIL import Image

# for visualizations

import matplotlib.pyplot as plt

import numpy as np # for using np arrays

# for bulding and running deep learning model
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split

In [3]:
def LoadData (input_dir, target_dir):
    """
    Looks for relevant filenames in the shared path
    Returns 2 lists for original and masked files respectively

    """
    input_img_paths =sorted([os.path.join(input_dir , fname )for fname in os.listdir(input_dir) if fname.endswith(".jpg")])
    target_img_paths = sorted([os.path.join(target_dir , fname) for fname in os.listdir(target_dir) if fname.endswith(".png") and not fname.startswith(".")])
    # Read the images folder like a list


    return input_img_paths, target_img_paths

In [4]:
def PreprocessData(img, mask, target_shape_img, target_shape_mask, path1, path2):
    """
    Processes the images and mask present in the shared list and path
    Returns a NumPy dataset with images as 3-D arrays of desired size
    Please note the masks in this dataset have only one channel
    """
    # Pull the relevant dimensions for image and mask
    m = len(img)                     # number of images
    i_h,i_w,i_c = target_shape_img   # pull height, width, and channels of image
    m_h,m_w,m_c = target_shape_mask  # pull height, width, and channels of mask

    # Define X and Y as number of images along with shape of one image
    X = np.zeros((m,i_h,i_w,i_c), dtype=np.float32)
    y = np.zeros((m,m_h,m_w,m_c), dtype=np.int32)

    # Resize images and masks
    for file in img:
        # convert image into an array of desired shape (3 channels)
        index = img.index(file)
        path = os.path.join(path1, file)
        single_img = Image.open(path).convert('RGB')
        single_img = single_img.resize((i_h,i_w))
        single_img = np.reshape(single_img,(i_h,i_w,i_c))
        single_img = single_img/256.
        X[index] = single_img

        # convert mask into an array of desired shape (1 channel)
        single_mask_ind = mask[index]
        path = os.path.join(path2, single_mask_ind)
        single_mask = Image.open(path)
        single_mask = single_mask.resize((m_h, m_w))
        single_mask = np.reshape(single_mask,(m_h,m_w,m_c))
        single_mask = single_mask - 1 # to ensure classes #s start from 0
        y[index] = single_mask
    return X, y

In [5]:
img_size = (128 , 128)
def path_to_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img , channels=3)
    img = tf.image.resize(img , img_size)
    img = tf.cast(img , tf.float32)
    return img
def path_to_target(path):
    img = tf.io.read_file(path )
    img = tf.image.decode_png(img ,channels=1 )
    img = tf.image.resize(img , img_size)
    img = tf.cast(img , tf.uint8) - 1
    return img
def map_fn(img_path , target_path):
    img = path_to_img(img_path)
    mask = path_to_target(target_path)
    return img , mask
num_valid_samples =1000

In [11]:
def EncoderMiniBlock(inputs, n_filters=32, dropout_prob=0.3, max_pooling=True,name="name"):
    """
    This block uses multiple convolution layers, max pool, relu activation to create an architecture for learning.
    Dropout can be added for regularization to prevent overfitting.
    The block returns the activation values for next layer along with a skip connection which will be used in the decoder
    """
    conv = Conv2D(n_filters,
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal', name = name+"1")(inputs)
    conv = Conv2D(n_filters,
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal', name = name+"2")(conv)

    conv = BatchNormalization()(conv, training=False)
    if dropout_prob > 0:
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)
    if max_pooling:
        next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)
    else:
        next_layer = conv
    skip_connection = conv

    return next_layer, skip_connection
def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=32, name="name"):
    """
    Decoder Block first uses transpose convolution to upscale the image to a bigger size and then,
    merges the result with skip layer results from encoder block
    Adding 2 convolutions with 'same' padding helps further increase the depth of the network for better predictions
    The function returns the decoded layer output
    """
    up = Conv2DTranspose(
                n_filters,
                (3,3),    # Kernel size
                strides=(2,2),
                padding='same',name=name+"transpose")(prev_layer_input)

    conv = Conv2D(n_filters,
                3,     # Kernel size
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal', name = name +"1")(up)
    conv = Conv2D(n_filters,
                3,   # Kernel size
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal', name = name +"2")(conv)
    return conv

def UNetCompiled(input_size=(128, 128, 3), n_filters=32, n_classes=3):

    inputs = Input(input_size)

    cblock1 = EncoderMiniBlock(inputs, n_filters,dropout_prob=0, max_pooling=True, name = "cblock1")
    cblock2 = EncoderMiniBlock(cblock1[0],n_filters*2,dropout_prob=0, max_pooling=True,name = "cblock2")
    cblock3 = EncoderMiniBlock(cblock2[0], n_filters*4,dropout_prob=0, max_pooling=True, name = "cblock3")
    cblock4 = EncoderMiniBlock(cblock3[0], n_filters*8,dropout_prob=0.3, max_pooling=True,name = "cblock4")
    cblock5 = EncoderMiniBlock(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False,name="cblock5")

    ublock6 = DecoderMiniBlock(cblock5[0], cblock4[1],  n_filters * 8, name="ublock6")
    ublock7 = DecoderMiniBlock(ublock6, cblock3[1],  n_filters * 4,name="ublock7")
    ublock8 = DecoderMiniBlock(ublock7, cblock2[1],  n_filters * 2,name="ublock8")
    ublock9 = DecoderMiniBlock(ublock8, cblock1[1],  n_filters,name="ublock9")
    conv9 = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='he_normal',name="conv9")(ublock9)

    conv10 = Conv2D(n_classes, 1, padding='same',name="conv10")(conv9)

    # Define the model
    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model
model= UNetCompiled(input_size=(128, 128, 3), n_filters=32, n_classes=3)

In [12]:
layer_names = [
 'cblock11',
 'cblock12',
 'cblock21',
 'cblock22',
 'cblock31',
 'cblock32',
 'cblock41',
 'cblock42',
 'cblock51',
 'cblock52',
 'ublock6transpose',
 'ublock61',
 'ublock62',
 'ublock7transpose',
 'ublock71',
 'ublock72',
 'ublock8transpose',
 'ublock81',
 'ublock82',
 'ublock9transpose',
 'ublock91',
 'ublock92',
 'conv9',
 'conv10']

In [13]:
# Load images and masks
path1 = '/content/drive/MyDrive/segmentation/images'
path2 = '/content/drive/MyDrive/segmentation/annotations/trimaps'
img, mask = LoadData(path1, path2)  # LoadData should return lists of image and mask paths


In [14]:
def set_trainable_layers(model, layers_to_train):
    """
    Set only specific layers to be trainable.

    Args:
    model: The model whose layers are to be modified.
    layers_to_train: A list of layer names to be set as trainable.
    """
    for layer in model.layers:
        if layer.name in layers_to_train:
            layer.trainable = True
        else:
            layer.trainable = False


In [15]:
import numpy as np
import tensorflow as tf
import random

# Number of clients
num_clients = 5

# Shuffle the dataset and divide it into num_clients parts
client_data_indices = np.array_split(np.arange(len(img)), num_clients)

# Create datasets for each client
client_datasets = []
for indices in client_data_indices:
    client_images = np.array(img)[indices]
    client_masks = np.array(mask)[indices]

    # Create TensorFlow datasets for each client
    client_dataset = tf.data.Dataset.from_tensor_slices((client_images, client_masks))
    client_dataset = client_dataset.map(map_fn).batch(64).prefetch(1)
    client_datasets.append(client_dataset)


In [ ]:
import random
global_model = UNetCompiled(input_size=(128, 128, 3), n_filters=32, n_classes=3)
all_layer_names = [layer.name for layer in global_model.layers]

def train_client_model(client_dataset, client_id, layers_to_train):
    local_model = tf.keras.models.clone_model(global_model)
    local_model.set_weights(global_model.get_weights())  # Copy global weights to the local model
    set_trainable_layers(local_model, layers_to_train)
    local_model.compile(optimizer=tf.keras.optimizers.Adam(),
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=['accuracy'])
    local_model.fit(client_dataset, epochs=1)
    print(f"Client {client_id} finished training.")
    return local_model.get_weights(), layers_to_train


In [ ]:
def federated_averaging(client_weights, client_layers):
    new_weights = [np.zeros_like(w) for w in client_weights[0]]
    for client_weight, layers_to_train in zip(client_weights, client_layers):
        for i, layer_name in enumerate(all_layer_names):
            if layer_name in layers_to_train:
                new_weights[i] += client_weight[i]
    new_weights = [w / len(client_weights) if all_layer_names[i] in set(sum(client_layers, [])) else w
                   for i, w in enumerate(new_weights)]
    return new_weights


In [ ]:
num_rounds = 10

for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    layers_to_train = random.sample(all_layer_names, 4)  # Example: select 4 random layers
    layers_to_train.extend(["conv9", "conv10"])  # Always train specific layers
    client_weights = []
    client_layers = []
    for client_id, client_dataset in enumerate(client_datasets):
        client_weight, trained_layers = train_client_model(client_dataset, client_id, layers_to_train)
        client_weights.append(client_weight)
        client_layers.append(trained_layers)

    avg_weights = federated_averaging(client_weights, client_layers)
    global_model.set_weights(avg_weights)

    print(f"Completed Round {round_num + 1}. Global model updated with layers: {layers_to_train}")
